# Excerise 4

In [1]:
import numpy as np
import pandas as pd
import os
import ast
import git
from git import RemoteProgress
from git import Repo
from datetime import datetime, date
import matplotlib.pyplot as plt
import seaborn as sns
from time import strptime
%matplotlib inline

In [2]:
class Progress(RemoteProgress):
    def update(self, op_code, cur_count, max_count=None, message=''):
        print(self._cur_line)

# Git Repo

In [3]:
#1
#remote_link = "https://github.com/apache/struts"
#local_link = "/home/amruta/Downloads/struts"
#repo = Repo(local_link)
#fixing_commit = "3fddfb6eb562d597c935084e9e81d43ed6bcd02"
#affected_files =["xwork-core/src/main/java/com/opensymphony/xwork2/validator/validators/URLValidator.java"]

#2
#remote_link = "https://github.com/cloudfoundry/uaa"
#local_link = "/home/amruta/Downloads/uaa"
#repo = Repo(local_link)
#fixing_commit = "7db5e5846961e08295b1ef7af909f267eebe5da"
#affected_files =["common/src/test/java/org/cloudfoundry/identity/uaa/zone/event/IdentityProviderModifiedEventTest.java"]
#affected_files =["common/src/test/java/org/cloudfoundry/identity/uaa/zone/event/IdentityZoneModifiedEventTest.java"]

#3
#remote_link = "https://github.com/apache/tomcat80"
#local_link = "/home/amruta/Downloads/tomcat80"
#repo = Repo(local_link)
#fixing_commit = "d752a415a875e888d8c8d0988dfbde95c2c6fb1d"
#vcc_commit=d8b8d4e8496cc8fc5b13833558bec61bdbda6ad1
#affected_files =["java/org/apache/tomcat/util/http/fileupload/MultipartStream.java"]
#affected_files =["webapps/docs/changelog.xml"]

#4
#remote_link = "https://github.com/spring-projects/spring-webflow"
#local_link = "/home/amruta/Downloads/spring-webflow"
#repo = Repo(local_link)
#fixing_commit = "57f2ccb66946943fbf3b3f2165eac1c8eb6b1523"
#affected_files = ["spring-webflow/src/main/java/org/springframework/webflow/mvc/view/AbstractMvcView.java"]

#5
#remote_link = "https://github.com/pingidentity/ldapsdk"
#local_link = "/home/amruta/Downloads/ldapsdk"
#repo = Repo(local_link)
#fixing_commit = "8471904a02438c03965d21367890276bc25fa5a6"
#affected_files =["docs/release-notes.html"]
#affected_files =["src/com/unboundid/ldap/sdk/SimpleBindRequest.java"]
#affected_files =["tests/unit/src/com/unboundid/ldap/sdk/SimpleBindRequestTestCase.java"]

#6
remote_link = "https://github.com/jenkinsci/jenkins"
local_link = "/home/amruta/Downloads/jenkins"
repo = Repo(local_link)
fixing_commit = "d7ea3f40efedd50541a57b943d5f7bbed046d091"
#affected_files = ["core/src/main/java/hudson/slaves/CommandConnector.java"]
#affected_files = ["core/src/main/java/hudson/slaves/CommandLauncher.java"]
#affected_files = ["core/src/main/resources/hudson/slaves/Messages.properties"]
#affected_files = ["test/src/test/java/hudson/model/ProjectTest.java"]
affected_files = ["test/src/test/java/hudson/slaves/CommandLauncher2Test.java"]


# Function findStartEnd

In [4]:
def StartEnd(file, line_affected):
    start = -1
    end = -1
    current_line = 1
    for line in file:
        if current_line == line_affected:
            i = current_line
            found = False
            while not found:
                i = i - 1
                if i == -1:
                    break
                if "{" in file[i - 1]:
                    found = True
                    start = i
                    break;
            found = False
            i = current_line
            while not found:
                i = i + 1
                if i >= (len(file) - 1):
                    break
                if "}" in file[i - 1]:
                    found = True
                    end = i
                    break   
        current_line = current_line + 1
    return start, end   

# Function makeList

In [5]:
def makeList(input_list, value):
    found = False
    for item in input_list:
        if item["commit"] == value:
            item["count"] = item["count"] + 1
            found = True
    if not found:
        input_list.append({"commit": value, "count": 1})
    return input_list

# Function findAddDelete

In [6]:
def AddDelete(line):
    if line[0] == "@@":
        delete = line[1]
        delete = delete[1:]
        delete = delete.split(",")
        if len(delete) < 2:
            delete.append("1")
        
        add = line[2]
        add = add[1:]
        add = add.split(",")
        if len(add) < 2:
            add.append("1")
                
        return add, delete

# Function findCommon

In [7]:
def Common(commits_found, exclude):
    maximum = -1
    commit_found = None
    for commit in commits_found:
        if exclude in commit["commit"]:
            continue
        if commit["count"] > maximum:
            commit_found = commit["commit"]
            maximum = commit["count"]
            
    return commit_found, maximum

# Function getVCC

In [8]:
def getVCC(files, parameters):
    for file in files:
        diff_of_file = repo.git.diff(fixing_commit + "^" + ".." + fixing_commit, "-U0", "--", file).splitlines()
        diff_of_file = diff_of_file[4:]
        raw_commit_file = repo.git.show(fixing_commit + ":" + file).splitlines()
        commits_found = []
        i = 0
        while i < len(diff_of_file):
            split_line = diff_of_file[i].split()
            if split_line[0] == "@@":
                additions, deletions = AddDelete(split_line)
                if deletions[1] != 0:
                    j = 0
                    line_number = int(deletions[0])
                    while j < int(deletions[1]):
                        j = j + 1
                        affected_file_diff = repo.git.blame(parameters,"-l", "-L" + str(line_number) + "," + str(line_number), fixing_commit, "--", file).splitlines()
                        split_line = affected_file_diff[0].split()
                        makeList(commits_found, split_line[0])
                    i = i + j
                if additions[1] != 0:
                    j = 0
                    line_number = int(additions[0])
                    while j < int(additions[1]):
                        j = j + 1
                        start, end = StartEnd(raw_commit_file, j + line_number)                         
                        if start != -1 and end != -1:
                            affected_file_diff = repo.git.blame(parameters,"-l", "-L" + str(start) + "," + str(end), fixing_commit, "--", file).splitlines()                         
                        else:
                            affected_file_diff = repo.git.blame(parameters,"-l", "-L" + str(line_number) + "," + str(line_number), fixing_commit, "--", file).splitlines()             
                        for line in affected_file_diff:
                                split_line = line.split()
                                makeList(commits_found, split_line[0])
                        line_number = line_number + 1
                    i = i + j                
            i = i + 1
    return Common(commits_found, fixing_commit)

## Get VCC ID

In [9]:
parameters = None
VCC = getVCC(affected_files, parameters)
print("VCC Commit:", VCC[0])

('VCC Commit:', None)
